In [2]:
using LinearAlgebra

function cross_prod_matrix( k )
    return [ 0  -k[3] k[2];  
            k[3] 0   -k[1]; 
           -k[2] k[1] 0 ]
end 

cross_prod_matrix (generic function with 1 method)

In [3]:
function rot_matrix_3d_transform( k, theta )
    mat0 = [ 1 0          0; 
             0 cos(theta) -sin(theta); 
             0 sin(theta) cos( theta )]
    
    if isapprox( k[1], 0.0 ) && isapprox( k[2], 0.0 )
        u = [ 0.0, 0.0, 1.0 ]
    else 
        u = [ k[2], -k[1], 0 ]
        u /= norm( u )
    end 
    
    v = cross( k, u )
    trans = hcat( k, u, v )
    return trans*mat0*transpose( trans )
end 

rot_matrix_3d_transform (generic function with 1 method)

In [4]:
function rot_matrix_3d( k, theta )
    
    K = cross_prod_matrix( k )
    return I + sin(theta)K + (1-cos(theta))K^2
end 

rot_matrix_3d (generic function with 1 method)

In [5]:
function rot_matrix_infinitesimal_3d( k, theta )
    K = cross_prod_matrix( k )
    return I+theta*K
end

rot_matrix_infinitesimal_3d (generic function with 1 method)

In [6]:
rot_x_theta( theta ) = rot_matrix_3d( [1.0, 0.0, 0.0], theta )
rot_y_theta( theta ) = rot_matrix_3d( [0.0, 1.0, 0.0], theta )
rot_z_theta( theta ) = rot_matrix_3d( [0.0, 0.0, 1.0], theta )    

rot_z_theta (generic function with 1 method)

In [7]:
function rand_rot_3d()

    k = [ rand()- .5 for x in 1:3 ]
    k = k/norm(k)
    theta = 2*pi*rand()
    return rot_matrix_3d( k, theta )
end

rand_rot_3d (generic function with 1 method)

In [8]:
function params_rotation_3d( R )
   
    theta = acos( (R[1,1] + R[2,2] + R[3,3] -1 )/2 )
    k1 = (R[3,2]-R[2,3])/(2*sin(theta))
    k2 = (R[1,3]-R[3,1])/(2*sin(theta))
    k3 = (R[2,1]-R[1,2])/(2*sin(theta))
    
    return [k1, k2, k3], theta
end

params_rotation_3d (generic function with 1 method)

In [9]:
function zxz_angles( R ) 
    
    theta = acos( R[3,3] )
    
    phi = pi/2#asin( R[3,1]/sin(theta) ); 
    if !isapprox( R[3,2], sin(theta)*cos(phi))
            phi = pi-phi
    end
    
    psi = -pi/2#asin( R[1,3]/sin(theta))
    if !isapprox( R[2,3], -cos(psi)*sin(theta)) 
        psi = pi-psi
    end 
    
    return phi, theta, psi
end

zxz_angles (generic function with 1 method)

In [10]:
function zyx_angles( R )
    theta = asin( -R[3,1] )
    phi = asin( R[3,2]/cos(theta))
    
    if !isapprox( cos(phi)*cos(theta), R[3,3] )
        phi = pi-phi
    end 
    
    psi = asin( R[2,1]/cos(theta))
    
    if !isapprox( cos(theta)*cos(psi), R[1,1])
        psi = pi-psi
    end
    
    return phi, theta, psi
end

zyx_angles (generic function with 1 method)

In [11]:
R = rand_rot_3d()

3×3 Matrix{Float64}:
  0.844943  -0.517754  -0.134169
  0.523493   0.749118   0.405929
 -0.109663  -0.413223   0.904003

In [12]:
phi, theta, psi = zyx_angles( R )
rot_z_theta( psi )*rot_y_theta( theta )*rot_x_theta( phi )

3×3 Matrix{Float64}:
  0.844943  -0.517754  -0.134169
  0.523493   0.749118   0.405929
 -0.109663  -0.413223   0.904003

In [13]:
phi, theta, psi = zxz_angles( R )
rot_z_theta( psi )*rot_x_theta( theta )*rot_z_theta( phi )

3×3 Matrix{Float64}:
  0.904003     3.22409e-16  -0.427527
 -3.11751e-16  1.0           9.493e-17
  0.427527     4.7465e-17    0.904003

In [14]:
r = rot_y_theta( 2pi*rand() )

3×3 Matrix{Float64}:
  0.70446   0.0  0.709744
  0.0       1.0  0.0
 -0.709744  0.0  0.70446

In [15]:
phi, theta, psi = zxz_angles( r )
rot_z_theta( psi )*rot_x_theta( theta )*rot_z_theta( phi )

3×3 Matrix{Float64}:
  0.70446      3.00255e-16  -0.709744
 -2.67444e-16  1.0           1.57595e-16
  0.709744     7.87974e-17   0.70446

In [16]:
isapprox(1.0000000000000002, 1 )

true

In [17]:
phi, theta, psi

(1.5707963267948966, 0.7891349991224235, 4.71238898038469)

In [18]:
pi/2

1.5707963267948966

In [19]:
k =[ 2*(rand()-.5) for i in 1:3 ]
theta = 0.001
r1 = rot_matrix_3d( k, theta )

3×3 Matrix{Float64}:
  1.0           0.000214264  0.000734224
 -0.000214369   1.0          0.000143978
 -0.000734193  -0.000144136  1.0

In [20]:
r2 = rot_matrix_infinitesimal_3d( k, theta )

3×3 Matrix{Float64}:
  1.0           0.000214316  0.000734208
 -0.000214316   1.0          0.000144057
 -0.000734208  -0.000144057  1.0

In [21]:
det(r1)

0.9999999999999402

In [22]:
det(r2)

1.0000006057459743